In [ ]:
import pandas as pd
import nltk
tweets=pd.read_csv('Tweets.csv')
tweets.head()

tweets.isna().sum()

In [ ]:
#delete columns which has lots of missing data
del tweets['tweet_coord']
del tweets['airline_sentiment_gold']
del tweets['negativereason_gold']
tweets.shape

In [ ]:
airlines= ['US Airways','United','American','Southwest','Delta','Virgin America']
plt.figure(1,figsize=(12, 12))
for i in airlines:
    indices= airlines.index(i)
    
    plt.subplot(2,3,indices+1)
    new_df=tweets[tweets['airline']==i]
    count=new_df['airline_sentiment'].value_counts()
    Index = [1,2,3]
    plt.bar(Index,count, color=['red', 'green', 'blue'])
    plt.xticks(Index,['negative','neutral','positive'])
    plt.ylabel('Mood Count')
    plt.xlabel('Mood')
    plt.title('Count of Moods of '+i)

# Data Pre Processing

In [ ]:
tweets_df=tweets.drop(tweets[tweets['airline_sentiment_confidence']<0.5].index,axis=0)
tweets_df.shape


# Clean Your Text Data

In [ ]:
from nltk.corpus import stopwords
nltk.download('stopwords')
import string
from nltk.stem import PorterStemmer


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
stop_words=stopwords.words('english')
punct=string.punctuation
stemmer=PorterStemmer()

In [ ]:
X=tweets_df['text']
y=tweets_df['airline_sentiment']

In [ ]:
import re
cleaned_data=[]
for i in range(len(X)):
  tweet=re.sub('[^a-zA-Z]',' ',X.iloc[i])
  tweet=tweet.lower().split()
  tweet=[stemmer.stem(word) for word in tweet if (word not in stop_words) and (word not in punct)]
  tweet=' '.join(tweet)
  cleaned_data.append(tweet)



In [ ]:
cleaned_data

In [ ]:
y

In [ ]:
sentiment_ordering = ['negative', 'neutral', 'positive']

y = y.apply(lambda x: sentiment_ordering.index(x))
print(y)

In [ ]:
y.head()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=3000,stop_words=['virginamerica','united','unit'])
X_fin=cv.fit_transform(cleaned_data).toarray()
X_fin.shape


In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X_fin,y,test_size=0.3)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dec_tree=DecisionTreeClassifier(random_state=0, max_depth=2)
dec_tree = dec_tree.fit(X_train, y_train)

In [ ]:
dec_log_predicted=dec_tree.predict(X_test)

In [ ]:
import sklearn.metrics
from mlxtend.plotting import plot_confusion_matrix

from sklearn.metrics import classification_report, confusion_matrix
cf=classification_report(y_test,dec_log_predicted)
cm=confusion_matrix(y_test,dec_log_predicted)
plt.figure()
plot_confusion_matrix(cm,figsize=(12,8), hide_ticks=True,cmap=plt.cm.Greens)
plt.show()
print(cf)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
model=MultinomialNB()

In [ ]:
model.fit(X_train,y_train)

In [ ]:
y_pred=model.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report
from mlxtend.plotting import plot_confusion_matrix
cf=classification_report(y_test,y_pred)
cm=confusion_matrix(y_test,y_pred)
plt.figure()
plot_confusion_matrix(cm,figsize=(12,8), hide_ticks=True,cmap=plt.cm.Blues)
plt.show()
print(cf)